In [1]:
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json
import datetime
import time
import numpy as np
import pandas as pd
import schedule
import keys

In [2]:
def get_date(date):
    year = int(date[0:4])
    month = int(date[5:7])
    day = int(date[8:10])
    return datetime.date(year,month,day)

In [3]:
def get_time(time):
    hour = int(time[11:13])
    minute = int(time[14:16])
    second = int(time[17:19])
    return datetime.time(hour,minute,second)

In [4]:
def get_age(date):
    now = datetime.datetime.now().date()
    return (now.year - date.year) * 12 + now.month - date.month

In [5]:
def get_listDF():
    with open('listDF.txt', 'r', encoding="utf-8") as f:
        listDF = eval(f.read())
    return listDF

In [6]:
def save_listDF(listDF):
    with open('listDF.txt', 'w+', encoding="utf-8") as f:
        f.write(str(listDF))

In [7]:
def get_job():
    with open('job.txt', 'r', encoding="utf-8") as f:
        job = eval(f.read())
    return job

In [8]:
def save_job(job):
    with open('job.txt', 'w+', encoding="utf-8") as f:
        f.write(str(job))

In [9]:
columns = ['id', 'rank', 'name', 'ticker', 'price', 'price_1h', 'price_24h', 'price_7d', 'price_30d', 'price_90d', 'marketcap', 'dominance', 'diluted_cap', 'volume24h', 'volume24_dollar', 'volume24_token', 'volume24_mrkt_pct', 'circulating', 'circulating_w_missing', 'max_supply', 'pairs', 'tags', 'platform', 'date_added', 'age', 'update_date', 'update_time']

In [10]:
#row = [ID, rank, name, ticker, price, pricepercent_1h, pricepercent_24h, pricepercent_7d, pricepercent_30d, pricepercent_90d, marketcap, marketcap_dominance, diluted_marketcap, volume24_percent, volume24_money, volume24_token, volume24_market_percent, circulating, circulating_w_missing, max_supply, pairs, tag, platform, date_added, age, update_date, update_time]

In [10]:
firstRow = [np.zeros(27)]

In [11]:
job = []
listDF = []

In [12]:
def pull_job():
    
    #Rank 1 - 2500
    url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'
    parameters = {
      'start':'1',
      'limit':'2500',
      'convert':'USD'
    }
    headers = {
      'Accepts': 'application/json',
      'X-CMC_PRO_API_KEY': keys.api_key,
    }

    session = Session()
    session.headers.update(headers)

    try:
        response = session.get(url, params=parameters)
        data = json.loads(response.text)
        global job

        for attribute in data['data']:
            ID = attribute['id']
            rank = attribute['cmc_rank']
            name = attribute['name']
            ticker = attribute['symbol']
            pairs = attribute['num_market_pairs']
            date_added = get_date(attribute['date_added'])
            age = get_age(date_added)
            tag = attribute['tags']
            max_supply = attribute['max_supply']
            circulating = round(attribute['circulating_supply'])
            circulating_w_missing = round(attribute['total_supply'])
            if max_supply != None:
                if max_supply != 0:
                    max_supply = round(max_supply)
                    supply_ratio = round(circulating_w_missing / max_supply,3)
            else:
                max_supply = 0
                supply_ratio = None
            platform = attribute['platform']
            if platform != None:
                platform = attribute['platform']['name']
            else:
                platform = ''
            updateDateTime = attribute['last_updated']
            update_date = get_date(updateDateTime)
            update_time = get_time(updateDateTime)

            #statics
            price = attribute['quote']['USD']['price']
            if price < 1:
                if round(price, 10) == 0:
                    price = round(price,20)
                else:
                    price = round(price,10)
            else:
                price = round(price,2)
            marketcap = round(attribute['quote']['USD']['market_cap'])
            volume24_money = round(attribute['quote']['USD']['volume_24h'])
            volume24_token = round((volume24_money / price))
            if marketcap > 0:
                volume24_market_percent = round((volume24_money / marketcap),3)
            else:
                volume24_market_percent = 0

            #changes
            volume24_percent = round(attribute['quote']['USD']['volume_change_24h'],2)
            pricepercent_1h = round(attribute['quote']['USD']['percent_change_1h'],2)
            pricepercent_24h = round(attribute['quote']['USD']['percent_change_24h'],2)
            pricepercent_7d = round(attribute['quote']['USD']['percent_change_7d'],2)
            pricepercent_30d = round(attribute['quote']['USD']['percent_change_30d'],2)
            pricepercent_90d = round(attribute['quote']['USD']['percent_change_90d'],2)
            marketcap_dominance = round(attribute['quote']['USD']['market_cap_dominance'],3)
            diluted_marketcap = round(attribute['quote']['USD']['fully_diluted_market_cap'])

            row = [ID, rank, name, ticker, price, pricepercent_1h, pricepercent_24h, pricepercent_7d, pricepercent_30d, pricepercent_90d, marketcap, marketcap_dominance, diluted_marketcap, volume24_percent, volume24_money, volume24_token, volume24_market_percent, circulating, circulating_w_missing, max_supply, pairs, tag, platform, date_added, age, update_date, update_time]
            job.append(row)
            row = []

        #timezone difference
        myTime = datetime.datetime.now().strftime("%d/%m/%Y, %H:%M:%S")
        CMCTime = datetime.datetime(update_date.year, update_date.month, update_date.day, update_time.hour, update_time.minute, update_time.second).strftime("%d/%m/%Y, %H:%M:%S")
        print('Hourly pull at:\nMy Time: {}\nCMC Time: {}'.format(myTime, CMCTime))
        print('len(job): {}'.format(len(job)))
        print('----------------------------------')
        dictionary = {'job': job, 'len(job)': len(job)}
        save_job(dictionary)
        
    except (ConnectionError, Timeout, TooManyRedirects) as e:
        print(e)

In [13]:
def update_db():
    global job
    global listDF
    global cmc_csv
    global df
    
    print('Updating DB:')
    print('len(job): {}'.format(len(job)))
    cmc_csv = pd.read_csv('cmc.csv')
    listDF = cmc_csv.values.tolist()
    listDF += job
    print('len(listDF): {}'.format(len(listDF)))    

    df = pd.DataFrame(data=listDF, columns=columns)
    print('df.shape: {}'.format(df.shape))
    df.to_csv('cmc.csv', index=False)
    job = []
    
    dictionary_job = {'job': job,'len(job)': len(job)}
    dictionary_listDF = {'listDF': listDF, 'len(listDF)': len(listDF)}
    save_job(dictionary_job)
    save_listDF(dictionary_listDF)
    
    print('Updated DB: {}'.format(datetime.datetime.now().strftime("%d/%m/%Y, %H:%M:%S")))
    print('--------------------------------------------')

In [14]:
len(job)

0

In [15]:
len(listDF)

0

In [16]:
pull_job()

Hourly pull at:
My Time: 03/08/2022, 00:59:26
CMC Time: 02/08/2022, 16:57:00
len(job): 2500
----------------------------------


In [17]:
update_db()

Updating DB:
len(job): 2500
len(listDF): 5001
df.shape: (5001, 27)
Updated DB: 03/08/2022, 00:59:29
--------------------------------------------


In [18]:
schedule.every().hour.at("02:00").do(pull_job)
schedule.every().day.at("07:30").do(update_db)

Every 1 day at 07:30:00 do update_db() (last run: [never], next run: 2022-08-03 07:30:00)

In [19]:
schedule.get_jobs()

[Every 1 hour at 00:02:00 do pull_job() (last run: [never], next run: 2022-08-03 01:02:00),
 Every 1 day at 07:30:00 do update_db() (last run: [never], next run: 2022-08-03 07:30:00)]

In [20]:
schedule.clear()

In [ ]:
# Loop so that the scheduling task
# keeps on running all time.
while True:
  
    # Checks whether a scheduled task 
    # is pending to run or not
    schedule.run_pending()
    time.sleep(1)